<a href="https://colab.research.google.com/github/HaiderMirza110/Agentic_AI/blob/main/Generative_AI_Projects/Project_02_LangChain_RAG_Project/LangChain%20_RAG_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain
!pip install pinecone-client
!pip install google-generativeai
!pip install openai
!pip install tqdm
!pip install langchain pinecone-client google-generativeai openai tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 7.3 MB/s eta 0:00:00


In [2]:
from google.colab import userdata
import os
PINECONE_API_KEY = userdata.get('PINECONE_API_KEY')
os.environ['PINECONE_API_KEY'] = PINECONE_API_KEY

GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
os.environ['GOOGLE_API_KEY'] = GOOGLE_API_KEY
PINECONE_ENVIRONMENT = 'us-east-1'

In [3]:
!pip install python-dotenv

In [4]:

import os
from dotenv import load_dotenv

load_dotenv()
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")


In [5]:
import os
from pinecone import Pinecone, ServerlessSpec


pc = Pinecone(
    api_key=PINECONE_API_KEY
)

# Check if the index exists; if not, create it
index_name = "gemini-1-5-flash"
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=768,
        metric="cosine",  # Choose the metric: cosine, euclidean, or dotproduct
        spec=ServerlessSpec(
            cloud="aws",
            region=PINECONE_ENVIRONMENT
        )
    )

# # Connect to the index
index = pc.Index(name=index_name)
print(f"Successfully connected to index: {index_name}")


Successfully connected to index: gemini-1-5-flash


In [6]:
!pip install langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 2.2 MB/s eta 0:00:00


In [7]:

from langchain_google_genai.embeddings import GoogleGenerativeAIEmbeddings

os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY # Assuming PINECONE_API_KEY is already defined


embeddings = GoogleGenerativeAIEmbeddings(
    model="models/embedding-001",  # Specify the desired embedding model
    api_key=GOOGLE_API_KEY
)

In [8]:
!pip install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 41.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.8 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.25
    Uninstalling langchain-core-0.3.25:
      Successfully uninstalled langchain-core-0.3.25
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.12
    Uninstalling langchain-0.3.12:
      Successfully uninstalled langchain-0.3.12


In [9]:

from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Load documents
loader = TextLoader("/content/History of Agentic AI.txt")  # Replace with your file
documents = loader.load()

# Split documents into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = text_splitter.split_documents(documents)

In [10]:
from tqdm import tqdm

# Create embeddings and upload to Pinecone
for doc in tqdm(docs):
    vector = embeddings.embed_query(doc.page_content)

    # Modify the upsert to use a dictionary for metadata
    index.upsert([{
        "id": doc.metadata["source"],  # Use "id" instead of the first tuple element
        "values": vector,  # Use "values" for the vector
        "metadata": {
            "text": doc.page_content,  # Add the text as part of metadata
            "source": doc.metadata["source"]  # Include the source
        }
    }])

100%|██████████| 9/9 [00:03<00:00,  2.96it/s]


In [11]:

from langchain.vectorstores import Pinecone

# Use from_existing_index to load from an existing index
retriever = Pinecone.from_existing_index(index_name=index_name, embedding=embeddings, text_key="text")

In [12]:
!pip install langchain-google-genai

In [13]:

from langchain_google_genai import ChatGoogleGenerativeAI

gemini_model = ChatGoogleGenerativeAI(api_key=GOOGLE_API_KEY,model="gemini-1.5-flash", temperature=0.7)

In [14]:
!pip install langchain-pinecone

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 12.8 MB/s eta 0:00:00
  Attempting uninstall: aiohttp
    Found existing installation: aiohttp 3.11.10
    Uninstalling aiohttp-3.11.10:
      Successfully uninstalled aiohttp-3.11.10


In [15]:
from langchain_pinecone import PineconeVectorStore
from langchain.chains import RetrievalQA

# Create a vector store using the Pinecone index
vectorstore = PineconeVectorStore(
    index_name=index_name,
    embedding=embeddings
)

# Create the retriever
retriever = vectorstore.as_retriever(search_kwargs={"k": 4})  # Retrieve top 4 most similar documents

# Create the QA chain
qa_chain = RetrievalQA.from_chain_type(
    llm=gemini_model,
    chain_type="stuff",
    retriever=retriever,  # Pass the retriever here
    return_source_documents=True  # Optional: to get the source documents used in the response
)


In [19]:
print("Welcome To Coding World")
print('='*40)

query = "What is Agentic Artificial Intelligence?"
print('Human Message:',query)
response = qa_chain.invoke(query)

# Print the answer
print("Agent Message:", response['result'])

# Print the source documents
print("\n According To Given Information:")
for doc in response['source_documents']:
    print(f"- {doc.page_content[:200]}...")  # Print first 200 characters of each source document

Welcome To Coding World
Human Message: What is Agentic Artificial Intelligence?
Agent Message: I'm sorry, but based on the provided text, I don't have enough information to define Agentic Artificial Intelligence.  The text only mentions that it's a field with ongoing research and development.


 According To Given Information:
- The history of Agentic AI is a continuous journey of innovation and discovery, with ongoing research and development pushing the boundaries of what is possible. As AI technology continues to advance, ...


In [17]:

print("Welcome To Coding World")
print('='*40)

query = "What is the Future of AI Agents?"
print('Human Message:',query)
response = qa_chain.invoke(query)

# Print the answer
print("Agent Message:", response['result'])

# Print the source documents
print("\n According To Given Information:")
for doc in response['source_documents']:
    print(f"- {doc.page_content[:200]}...")  # Print first 200 characters of each source document

Welcome To Coding World
Human Message: What is the Future of AI Agents?
Agent Message: Based on the provided text, the future of AI agents involves continued innovation, discovery, research, and development, leading to more sophisticated and capable systems.


 According To Given Information:
- The history of Agentic AI is a continuous journey of innovation and discovery, with ongoing research and development pushing the boundaries of what is possible. As AI technology continues to advance, ...
